# Test with github

This notebook will take the "validation" file and will test it against our AI lab model, in order to know how effective our model is.

In [1]:
from tqdm import tqdm

import requests
import pandas as pd
import numpy as np

In [2]:
validation = pd.read_csv("./validation.csv")


MODELS_V1 = {
    "ELMO - WOLI-120k-T0.3-NO_VAL - Logistic Regression using TF-IDF vectors V1" : {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/c9830d4c-9c75-11eb-98a5-de79e166b688",
        "accuracy": None,
        "predicted_values" : [],
    },
    "ELMO - WOLI-24k-T0.3-NO_VAL - Logistic Regression using TF-IDF vectors V1" : {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/105077b0-9c93-11eb-ba53-de79e166b688",
        "accuracy": None,
        "predicted_values" : [],
    },
    "ELMO - WOLI-24k-T0.3-VAL - Logistic Regression using TF-IDF vectors V1" : {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/1071863e-9c94-11eb-ba92-de79e166b688",
        "accuracy": None,
        "predicted_values" : [],
    },
    "ELMO - WOLI-240K-T0.3-NO_VAL - Logistic Regression using TF-IDF vectors V2 no stop words" : {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/6f575112-9ca6-11eb-aefd-de79e166b688",
        "accuracy": None,
        "predicted_values" : [],
    },
    "WOLI - Bert -120k FFNN using BERT vectors": {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/762877ca-9a79-11eb-8d07-de79e166b688",
        "accuracy": None,
        "predicted_values" : [],
    },
     "WOLI - DistilBert -120k FFNN using DistilBERT vectors": {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/35ae0026-9a0f-11eb-9a75-de79e166b688",
        "accuracy": None,
        "predicted_values" : [],
    },
    "ELMO - WOLI - Bert - 300/300 FFNN using BERT vectors no stop words": {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/30d948b8-9d4c-11eb-aa0f-521c6757c414",
        "accuracy": None,
        "predicted_values" : [],
    },   
    "ELMO - WOLI - Bert - 30k/30k FFNN using DistilBERT vectors no stop words": {
        "url": "https://predict-ailab.uruit.com/text/classification/predict/03279852-9d6a-11eb-a90b-8697a6fa86bd",
        "accuracy": None,
        "predicted_values" : [],
    },
    "ELMO - WOLI - Bert - 30k/30k FFNN using BERT vectors no stop words":{
        "url": "https://predict-ailab.uruit.com/text/classification/predict/49fb832a-9dfa-11eb-b2b9-8697a6fa86bd",
        "accuracy": None,
        "predicted_values" : [],
    },
    "ELMO - WOLI  Distilbert T3,0 100k/100k no stop words":{
        "url": "https://predict-ailab.uruit.com/text/classification/predict/721db1da-9eee-11eb-8df4-8697a6fa86bd",
        "accuracy": None,
        "predicted_values" : [],
    }
    
}

MODELS = {
    "ELMO - WOLI - Bert - 300/300 FFNN using BERT vectors no stop words": MODELS_V1["ELMO - WOLI - Bert - 300/300 FFNN using BERT vectors no stop words"],
    "ELMO - WOLI-240K-T0.3-NO_VAL - Logistic Regression using TF-IDF vectors V2 no stop words" : MODELS_V1["ELMO - WOLI-240K-T0.3-NO_VAL - Logistic Regression using TF-IDF vectors V2 no stop words"],
    "ELMO - WOLI - Bert - 30k/30k FFNN using DistilBERT vectors no stop words": MODELS_V1["ELMO - WOLI - Bert - 30k/30k FFNN using DistilBERT vectors no stop words"],
    "ELMO - WOLI - Bert - 30k/30k FFNN using BERT vectors no stop words": MODELS_V1["ELMO - WOLI - Bert - 30k/30k FFNN using BERT vectors no stop words"],
    "ELMO - WOLI  Distilbert T3,0 100k/100k no stop words": MODELS_V1["ELMO - WOLI  Distilbert T3,0 100k/100k no stop words"]
}

validation.head()


,Text,Label
0,ima be home tomorrow so line up a bitch for me,TOXIC
1,yall hoes still play around with the wrong shi...,TOXIC
2,binko denzy tryna hoe broski izzy,TOXIC
3,rt causewereguys flirting with bitches on a tw...,TOXIC
4,watching the nigger movie menacesociety trying...,TOXIC


In [3]:
allLen = len(validation)

SUB_SET_SIZE = 100

indexes = np.arange(allLen);
np.random.shuffle(indexes)
rndIdx = indexes[:SUB_SET_SIZE]
validation_sub_set = validation.iloc[rndIdx]
validation_sub_set.reset_index(inplace=True, drop=True)
validation_sub_set.head()

,Text,Label
0,jonjanke deadspin jtrimbl you sound like you r...,TOXIC
1,the trash wheel has a twitter and googly eyes,HEALTHY
2,the_red_sea shethetruththo say that shit to my...,TOXIC
3,good they trash rt jerseyzbest niggas ate most...,TOXIC
4,neva been a bitch nigga you don t want them pr...,TOXIC


In [4]:
def getAccuracy(predicted_values):
    errors_array = []
    for index, row in validation_sub_set.iterrows():
        text = row['Text']
        label = row['Label']
        predicted =  predicted_values[index]
        if (label != predicted == 'HEALTHY'):
            errors_array.append([text,label,predicted])

    errors_df = pd.DataFrame(errors_array,columns=['Text','Expected','Received'])
    errorsLen = len(errors_array)
    subsetLen = len(validation_sub_set)
    accuracy = 100 - ((errorsLen / subsetLen) * 100)
    return accuracy
    

In [5]:
items = MODELS.items();
allItemsLen = len(items) * len(validation_sub_set);
headers = {'accept':'application/json','Content-Type': 'application/json'}
pbar = tqdm(total=allItemsLen)
# response example {'processing_time': 0.024, 'confidence_score': 0.983, 'result': '1.0'}
for name, modelData in items:
    predicted_values = []
    for index, row in validation_sub_set.iterrows():
        url = modelData["url"]
        data = '{"text":"'+row['Text']+'"}'
        response = requests.post(url, headers=headers, data=data)
        response = response.json()
        predicted_value = response['result']
        
        if(predicted_value == "0.0"):
            predicted_value = 'HEALTHY'
        elif(predicted_value == "1.0"):
            predicted_value = 'TOXIC'
        
        predicted_values.append(predicted_value)
        pbar.update(n=1)
    MODELS[name]['accuracy'] = getAccuracy(predicted_values)
   


100%|██████████| 500/500 [04:01<00:00,  2.34it/s]

Now we will verify if our model is effective with our validation data

In [6]:
MODELS

{'ELMO - WOLI - Bert - 300/300 FFNN using BERT vectors no stop words': {'url': 'https://predict-ailab.uruit.com/text/classification/predict/30d948b8-9d4c-11eb-aa0f-521c6757c414',
  'accuracy': 73.0,
  'predicted_values': []},
 'ELMO - WOLI-240K-T0.3-NO_VAL - Logistic Regression using TF-IDF vectors V2 no stop words': {'url': 'https://predict-ailab.uruit.com/text/classification/predict/6f575112-9ca6-11eb-aefd-de79e166b688',
  'accuracy': 73.0,
  'predicted_values': []},
 'ELMO - WOLI - Bert - 30k/30k FFNN using DistilBERT vectors no stop words': {'url': 'https://predict-ailab.uruit.com/text/classification/predict/03279852-9d6a-11eb-a90b-8697a6fa86bd',
  'accuracy': 85.0,
  'predicted_values': []},
 'ELMO - WOLI - Bert - 30k/30k FFNN using BERT vectors no stop words': {'url': 'https://predict-ailab.uruit.com/text/classification/predict/49fb832a-9dfa-11eb-b2b9-8697a6fa86bd',
  'accuracy': 88.0,
  'predicted_values': []},
 'ELMO - WOLI  Distilbert T3,0 100k/100k no stop words': {'url': 'ht